In [2]:
import pandas as pd
import numpy as np
import concurrent.futures

In [3]:
#Global paramenters
#API URLs from RxNorm
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
base_url_3 = "https://rxnav.nlm.nih.gov/REST/rxcui.json?name="
#RxNorm methods
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
filterVet = "/filter.json?propName=RXNAV_VET_DRUG"
filterVet2 = "/filter.json?propName=VET_DRUG"
filterHuman = "/filter.json?propName=RXNAV_HUMAN_DRUG"
filterHuman2 = "/filter.json?propName=HUMAN_DRUG"
#dataframe
df = pd.DataFrame()

In [77]:
#Method to get drug with only the opioid ingredient
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    rd_df = rd_df.loc[rd_df["tty"] != "DF"].reset_index(drop=True)
    rd_df = rd_df.loc[rd_df["tty"] != "DFG"].reset_index(drop=True)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
            row += 1
    return df

ALL Opioid Ingredients

In [78]:
#Read in ingredients from excel file
drug_ingredients = pd.read_excel("opioid_Ingredients.xlsx")
drugs = []
#Get rxcui for each ingredient
for drug in drug_ingredients["Ingredient"]:
    print(drug)
    ingredient = pd.read_json(base_url_3+drug)
    drugs.append(ingredient.idGroup.rxnormId[0])
#loop array to get complete list of drugs
df_opioid = pd.DataFrame()
for drug in drugs:
    self_df = query_self(drug)
    ingredient = self_df["name"].values[0]
    self_df["ingredient"] = ingredient
    merg_df = query_drug(drug)
    merg_df["ingredient"] = ingredient
    df_opioid = pd.concat([df_opioid, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df_opioid = pd.concat([df_opioid, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_opioid

alfentanil
alphaprodine
Anileridine
benzhydrocodone
buprenorphine
butorphanol
codeine
dextromoramide
dezocine
difenoxin
dihydrocodeine
diphenoxylate
dipipanone
ethylmorphine
fentanyl
heroin
hydrocodone
hydromorphone
levomethadone
levomethadyl
levopropoxyphene
levorphanol
meperidine
meptazinol
methadone
methadyl+acetate
morphine
nalbuphine
normethadone
noscapine
oliceridine
opium
oxycodone
oxymorphone
papaveretum
paregoric
pentazocine
pethidine
phenazocine
phenoperidine
Pholcodine
pirinitramide
propoxyphene
remifentanil
sufentanil
tapentadol
tetrapeptide-15
tilidine
tramadol


,rxcui,name,synonym,tty,language,suppress,umlscui,ingredient
0,480,alfentanil,,IN,ENG,N,,alfentanil
1,203203,alfentanil hydrochloride,,PIN,ENG,N,,alfentanil
2,106500,alfentanil 5 MG/ML Injectable Solution,alfentanil (as alfentanil hydrochloride) 5 MG/...,SCD,ENG,N,,alfentanil
3,1723206,2 ML alfentanil 0.5 MG/ML Injection,alfentanil (as alfentanil HCl) 500 MCG/ML in 2...,SCD,ENG,N,,alfentanil
4,1723208,10 ML alfentanil 0.5 MG/ML Injection,alfentanil (as alfentanil HCl) 500 MCG/ML in 1...,SCD,ENG,N,,alfentanil
...,...,...,...,...,...,...,...,...
2730,1157662,tramadol Oral Product,,SCDG,ENG,N,,tramadol
2731,1157663,tramadol Pill,,SCDG,ENG,N,,tramadol
2732,1295674,tramadol Disintegrating Oral Product,,SCDG,ENG,N,,tramadol
2733,2588475,celecoxib / tramadol Oral Product,,SCDG,ENG,N,,tramadol


In [79]:
vet_rxcuis = []
def humanValidate(rxcui):
    global vet_rxcuis
    humanMedication = False
    if not pd.read_json(base_url + rxcui + filterHuman, typ="series").empty or \
        not pd.read_json(base_url + rxcui + filterHuman2, typ="series").empty:
        humanMedication = True

    if humanMedication is False:
        vet_rxcuis.append(rxcui)
        return 1  # Indicates a vet medication
    return 0

def process_rxcui(rxcui):
    if not pd.read_json(base_url + rxcui + filterVet, typ="series").empty or \
       not pd.read_json(base_url + rxcui + filterVet2, typ="series").empty:
        return humanValidate(rxcui)
    return 0

# Assuming df_opioid is a DataFrame with a column 'rxcui'
count = 0
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_rxcui, df_opioid['rxcui'])
    count = sum(results)  # Sum the results to get the total count

print(f"Total vet medications: {count}")
vet_rxcuis


Total vet medications: 41


['1594651',
 '2599847',
 '1594655',
 '2599851',
 '1594652',
 '2599848',
 '1594653',
 '2599850',
 '1594654',
 '2599849',
 '1594650',
 '2599846',
 '1594649',
 '2599844',
 '2599845',
 '379112',
 '1310923',
 '1489987',
 '2632847',
 '857190',
 '1302741',
 '1310927',
 '1809204',
 '2632851',
 '1302740',
 '1489988',
 '2632848',
 '1310924',
 '1310925',
 '1809203',
 '1489989',
 '2632850',
 '857192',
 '2655642',
 '1174579',
 '1310926',
 '1489990',
 '2632849',
 '1302739',
 '1489991',
 '1302738']

In [80]:
len(vet_rxcuis)

41

In [81]:
# Create df_vet from rows in df_opioid that have rxcuis in vet_rxcuis
df_vet = df_opioid[df_opioid['rxcui'].isin(vet_rxcuis)].copy()

# Remove these rows from df_opioid
df_opioid = df_opioid[~df_opioid['rxcui'].isin(vet_rxcuis)]

In [82]:
df_opioid.reset_index(drop=True, inplace=True)

In [83]:
df_vet

,rxcui,name,synonym,tty,language,suppress,umlscui,ingredient
41,1594651,Simbadol,,BN,ENG,N,,buprenorphine
46,2599847,Zorbium,,BN,ENG,N,,buprenorphine
63,1594655,buprenorphine 1.8 MG/ML Injectable Solution [S...,Simbadol 1.8 MG/ML Injectable Solution,SBD,ENG,N,,buprenorphine
79,2599851,buprenorphine 20 MG/ML Topical Solution [Zorbium],Zorbium 20 MG/ML Topical Solution,SBD,ENG,N,,buprenorphine
101,1594652,buprenorphine 1.8 MG/ML [Simbadol],,SBDC,ENG,N,,buprenorphine
115,2599848,buprenorphine 20 MG/ML [Zorbium],,SBDC,ENG,N,,buprenorphine
124,1594653,buprenorphine Injectable Solution [Simbadol],,SBDF,ENG,N,,buprenorphine
131,2599850,buprenorphine Topical Solution [Zorbium],,SBDF,ENG,N,,buprenorphine
143,1594654,Simbadol Injectable Product,,SBDG,ENG,N,,buprenorphine
149,2599849,Zorbium Topical Product,,SBDG,ENG,N,,buprenorphine


In [84]:
df_opioid

,rxcui,name,synonym,tty,language,suppress,umlscui,ingredient
0,480,alfentanil,,IN,ENG,N,,alfentanil
1,203203,alfentanil hydrochloride,,PIN,ENG,N,,alfentanil
2,106500,alfentanil 5 MG/ML Injectable Solution,alfentanil (as alfentanil hydrochloride) 5 MG/...,SCD,ENG,N,,alfentanil
3,1723206,2 ML alfentanil 0.5 MG/ML Injection,alfentanil (as alfentanil HCl) 500 MCG/ML in 2...,SCD,ENG,N,,alfentanil
4,1723208,10 ML alfentanil 0.5 MG/ML Injection,alfentanil (as alfentanil HCl) 500 MCG/ML in 1...,SCD,ENG,N,,alfentanil
...,...,...,...,...,...,...,...,...
2689,1157662,tramadol Oral Product,,SCDG,ENG,N,,tramadol
2690,1157663,tramadol Pill,,SCDG,ENG,N,,tramadol
2691,1295674,tramadol Disintegrating Oral Product,,SCDG,ENG,N,,tramadol
2692,2588475,celecoxib / tramadol Oral Product,,SCDG,ENG,N,,tramadol


In [85]:
#Reduce to usefull columns
df_opioid = df_opioid[['rxcui', 'name',"tty", "ingredient"]]
df_opioid

,rxcui,name,tty,ingredient
0,480,alfentanil,IN,alfentanil
1,203203,alfentanil hydrochloride,PIN,alfentanil
2,106500,alfentanil 5 MG/ML Injectable Solution,SCD,alfentanil
3,1723206,2 ML alfentanil 0.5 MG/ML Injection,SCD,alfentanil
4,1723208,10 ML alfentanil 0.5 MG/ML Injection,SCD,alfentanil
...,...,...,...,...
2689,1157662,tramadol Oral Product,SCDG,tramadol
2690,1157663,tramadol Pill,SCDG,tramadol
2691,1295674,tramadol Disintegrating Oral Product,SCDG,tramadol
2692,2588475,celecoxib / tramadol Oral Product,SCDG,tramadol


In [86]:
#Export
df_opioid.to_excel("New_Complete_Opioids_List.xlsx", index=False)

In [87]:
#take list and format it to the RXNorm api format
#This list was made by combinding the excel files
df_list = pd.read_excel("Complete_Opioids_List.xlsx")
df_list = df_list[['Code', 'Description','TTY']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name,TTY
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SCD
1,1010603,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SBD
2,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SCD
3,1010606,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SBD
4,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,SCD
...,...,...,...
2254,995842,Biotussin,BN
2255,2179635,tramadol hydrochloride 100 MG Oral Tablet,SCD
2256,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD
2257,864826,methadone hydrochloride 5 MG/ML Oral Solution,SCD


In [88]:
df_list["ingredients"] = None
for i, row in df_list.iterrows():
    try:
        df_active_ing = query_self(str(df_list.loc[i, "rxcui"]))
        df_list.at[i, 'ingredients'] = list(df_active_ing['name'])
    except AttributeError:
        continue

In [89]:
df_list['ingredients'] = df_list['ingredients'].fillna('Obsolete Error')
df_list

,rxcui,name,TTY,ingredients
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SCD,"[buprenorphine, naloxone]"
1,1010603,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SBD,"[buprenorphine, naloxone]"
2,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SCD,"[buprenorphine, naloxone]"
3,1010606,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SBD,"[buprenorphine, naloxone]"
4,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,SCD,"[acetaminophen, oxycodone]"
...,...,...,...,...
2254,995842,Biotussin,BN,"[codeine, guaifenesin, pseudoephedrine]"
2255,2179635,tramadol hydrochloride 100 MG Oral Tablet,SCD,[tramadol]
2256,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete Error
2257,864826,methadone hydrochloride 5 MG/ML Oral Solution,SCD,[methadone]


In [90]:
df_list

,rxcui,name,TTY,ingredients
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SCD,"[buprenorphine, naloxone]"
1,1010603,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SBD,"[buprenorphine, naloxone]"
2,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SCD,"[buprenorphine, naloxone]"
3,1010606,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SBD,"[buprenorphine, naloxone]"
4,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,SCD,"[acetaminophen, oxycodone]"
...,...,...,...,...
2254,995842,Biotussin,BN,"[codeine, guaifenesin, pseudoephedrine]"
2255,2179635,tramadol hydrochloride 100 MG Oral Tablet,SCD,[tramadol]
2256,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete Error
2257,864826,methadone hydrochloride 5 MG/ML Oral Solution,SCD,[methadone]


In [91]:
df_list.to_excel("Complete_Opioids_List_with_Ingredients.xlsx", index=False)

In [92]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame()
new_drug = pd.DataFrame()
for code in df_opioid['rxcui']:
    drug_name = df_opioid['name'].where(df_opioid['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = pd.concat([same_drug, df_opioid.loc[df_opioid['rxcui']==code]]).reset_index(drop=True)
    else:
        new_drug_count += 1
        new_drug = pd.concat([new_drug, df_opioid.loc[df_opioid['rxcui']==code]]).reset_index(drop=True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  1942
New drug count:  752


In [93]:
#export
same_drug.to_excel("Same_Opioid_Medications.xlsx", index=False)
new_drug.to_excel("New_Opioids_Added.xlsx", index=False)

In [94]:
absent_drugs = pd.DataFrame()
count = 0
#for every code from the combind excel list
for code in df_list['rxcui']:
    if df_opioid.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = pd.concat([absent_drugs, df_list.loc[df_list['rxcui']==code]]).reset_index(drop=True)
print('Count: ', count)
absent_drugs

Count:  1942


,rxcui,name,TTY,ingredients
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD,Obsolete Error
1,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...,SBD,Obsolete Error
2,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD,Obsolete Error
3,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD,Obsolete Error
4,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD,Obsolete Error
...,...,...,...,...
312,71225,apomorphine hydrochloride,PIN,[apomorphine]
313,89781,"belladonna extract, USP",IN,"[belladonna extract, USP]"
314,993764,Capital and Codeine,BN,Obsolete Error
315,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete Error


In [95]:
#find the status of the not found drugs
df_status = pd.DataFrame()
index = 0
for code in absent_drugs['rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url_2+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    df_status = pd.concat([df_status, df_type]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_status

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
0,Obsolete,RXNORM,012011,,NO,012011,122021,,1050409
1,Obsolete,RXNORM,022011,,NO,022011,022023,,1087389
2,Obsolete,RXNORM,072011,,NO,072011,052021,,1112224
3,Obsolete,RXNORM,082011,,NO,082011,122021,,1113998
4,Obsolete,RXNORM,082011,,NO,082011,122021,,1114002
...,...,...,...,...,...,...,...,...,...
312,Active,RXNORM,042005,,YES,042005,,,71225
313,Active,RXNORM,042005,,YES,042005,,,89781
314,Obsolete,RXNORM,072010,,NO,072010,072021,,993764
315,Obsolete,RXNORM,092019,,NO,092019,122021,,2182349


In [96]:
df_not_found = absent_drugs.merge(df_status, on='rxcui')

In [97]:
df_not_found = df_not_found[['rxcui', 'name','TTY', 'status', 'ingredients']]
df_not_found

,rxcui,name,TTY,status,ingredients
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD,Obsolete,Obsolete Error
1,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...,SBD,Obsolete,Obsolete Error
2,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD,Obsolete,Obsolete Error
3,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD,Obsolete,Obsolete Error
4,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD,Obsolete,Obsolete Error
...,...,...,...,...,...
312,71225,apomorphine hydrochloride,PIN,Active,[apomorphine]
313,89781,"belladonna extract, USP",IN,Active,"[belladonna extract, USP]"
314,993764,Capital and Codeine,BN,Obsolete,Obsolete Error
315,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete,Obsolete Error


In [98]:
df_active=df_not_found.loc[df_not_found['status']== 'Active'].reset_index(drop=True)
df_obsolete = df_not_found.loc[df_not_found['status']== 'Obsolete'].reset_index(drop=True)

In [99]:
#export
df_not_found.to_excel("Status_of_Not_found.xlsx", index=False)
df_active.to_excel("Active_Not_Found.xlsx", index=False)
df_obsolete.to_excel("Obsolete_Not_Found.xlsx", index=False)

In [32]:
#Testing move up later PLZ
df_nonpain = pd.read_excel("New_Complete_Opioids_List.xlsx")
df_nonpain

,rxcui,name,tty,ingredient
0,480,alfentanil,IN,alfentanil
1,203203,alfentanil hydrochloride,PIN,alfentanil
2,106500,alfentanil 5 MG/ML Injectable Solution,SCD,alfentanil
3,1723206,2 ML alfentanil 0.5 MG/ML Injection,SCD,alfentanil
4,1723208,10 ML alfentanil 0.5 MG/ML Injection,SCD,alfentanil
...,...,...,...,...
2689,1157662,tramadol Oral Product,SCDG,tramadol
2690,1157663,tramadol Pill,SCDG,tramadol
2691,1295674,tramadol Disintegrating Oral Product,SCDG,tramadol
2692,2588475,celecoxib / tramadol Oral Product,SCDG,tramadol


In [38]:
# Reading the Excel file (assuming you've already done this)
df_nonpain = pd.read_excel("New_Complete_Opioids_List.xlsx")

drug_ingredients = pd.read_excel("opioid_Ingredients.xlsx")

nonpainIngredients = drug_ingredients[(drug_ingredients['NON-Pain'] == "true")]
nonpainIngredients = nonpainIngredients['Ingredient'].to_numpy()

# Filtering the main DataFrame for non-pain ingredients
condition1 = df_nonpain['ingredient'].isin(nonpainIngredients)

# Separate condition for 'opium' ingredient
opium_condition = (df_nonpain['ingredient'].str.lower() == "opium")
opium_tincture_condition = opium_condition & \
                           df_nonpain['name'].str.contains("tincture", case=False, na=False)

df_nonpain = df_nonpain[(condition1 & ~opium_condition) | opium_tincture_condition]
df_nonpain

,rxcui,name,tty,ingredient
869,23024,difenoxin,IN,difenoxin
870,218445,Motofen,BN,difenoxin
871,214261,atropine / difenoxin,MIN,difenoxin
872,235373,difenoxin hydrochloride,PIN,difenoxin
873,1190740,atropine sulfate 0.025 MG / difenoxin hydrochl...,SBD,difenoxin
874,1190739,atropine sulfate 0.025 MG / difenoxin hydrochl...,SBDC,difenoxin
875,369383,atropine / difenoxin Oral Tablet [Motofen],SBDF,difenoxin
876,2656463,atropine sulfate / difenoxin hydrochloride Ora...,SBDFP,difenoxin
877,1182809,Motofen Oral Product,SBDG,difenoxin
878,1182810,Motofen Pill,SBDG,difenoxin


In [39]:
df_nonpain.reset_index(drop=True, inplace=True)
df_nonpain

,rxcui,name,tty,ingredient
0,23024,difenoxin,IN,difenoxin
1,218445,Motofen,BN,difenoxin
2,214261,atropine / difenoxin,MIN,difenoxin
3,235373,difenoxin hydrochloride,PIN,difenoxin
4,1190740,atropine sulfate 0.025 MG / difenoxin hydrochl...,SBD,difenoxin
5,1190739,atropine sulfate 0.025 MG / difenoxin hydrochl...,SBDC,difenoxin
6,369383,atropine / difenoxin Oral Tablet [Motofen],SBDF,difenoxin
7,2656463,atropine sulfate / difenoxin hydrochloride Ora...,SBDFP,difenoxin
8,1182809,Motofen Oral Product,SBDG,difenoxin
9,1182810,Motofen Pill,SBDG,difenoxin


In [40]:
df_nonpain.to_excel("non-painOpioids.xlsx", index=False)

In [25]:
df_new = pd.read_excel(r"C:\Users\Daniel\Documents\opiod_list\Atomatic_Code_Sets\RxNorm\Appendices\Appendix3.xlsx")

In [26]:
df_new

,rxcui,name,tty,ingredient
0,480,alfentanil,IN,alfentanil
1,203203,alfentanil hydrochloride,PIN,alfentanil
2,106500,alfentanil 5 MG/ML Injectable Solution,SCD,alfentanil
3,1723206,2 ML alfentanil 0.5 MG/ML Injection,SCD,alfentanil
4,1723208,10 ML alfentanil 0.5 MG/ML Injection,SCD,alfentanil
...,...,...,...,...
1937,1157660,tramadol Injectable Product,SCDG,tramadol
1938,1157661,tramadol Oral Liquid Product,SCDG,tramadol
1939,1157662,tramadol Oral Product,SCDG,tramadol
1940,1157663,tramadol Pill,SCDG,tramadol


In [27]:
for code in df_new['rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url_2+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    df_new_date = pd.concat([df_new_date, df_type]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)

In [28]:
df_new_date.drop(df_new_date.columns[0],axis=1,inplace=True)

In [29]:
df_new_date.drop(0, inplace=True)

In [62]:
df_new_date = pd.read_excel("CS7.xlsx")

In [63]:
df_new_date['activeStartDate'] = df_new_date['activeStartDate'].astype(str)
df_new_date = df_new_date[df_new_date['activeStartDate'].str[-4:] > '2015']
df_new_date

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
3,Active,RXNORM,12016,NaN,YES,12016,NaN,NaN,1723206
4,Active,RXNORM,12016,NaN,YES,12016,NaN,NaN,1723208
5,Active,RXNORM,12016,NaN,YES,12016,NaN,NaN,1723209
6,Active,RXNORM,12016,NaN,YES,12016,NaN,NaN,1723210
7,Active,RXNORM,12016,NaN,YES,12016,NaN,NaN,1723205
...,...,...,...,...,...,...,...,...,...
1914,Active,RXNORM,112017,NaN,YES,112017,NaN,NaN,1946525
1915,Active,RXNORM,112017,NaN,YES,112017,NaN,NaN,1946527
1916,Active,RXNORM,112017,NaN,YES,112017,NaN,NaN,1946529
1917,Active,RXNORM,82019,NaN,YES,82019,NaN,NaN,2179635
